# EWC

In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adamax
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.metrics import Mean
from tensorflow.keras.utils import to_categorical

In [2]:
import import_ipynb
import pandas as pd
import numpy as np

In [3]:
from copy import deepcopy

In [4]:
import utils
from utils import *

In [5]:
import Model 
from Model import *

## org

In [6]:
def compute_ewc_penalty(model, fisher_matrix, optimal_weights, lamb):   
    loss = 0
    current = model.trainable_weights 
    
    for F, c, o in zip(fisher_matrix, current, optimal_weights):
        loss += tf.reduce_sum(F * ((c - o) ** 2))


    return loss * (lamb / 2)

In [7]:
def ewc_loss(model, fisher_matrix, lamb):
    optimal_weights = deepcopy(model.trainable_weights)

    def loss_fn(y_true, y_pred):

        ce_loss = CategoricalCrossentropy(from_logits=False)(y_true, y_pred)
        ewc_loss = compute_ewc_penalty(model, fisher_matrix, optimal_weights, lamb=lamb)

        return ce_loss + ewc_loss
    
    return loss_fn

In [8]:
def compute_fisher_matrix(model, data, num_sample=10):

    weights = model.trainable_weights
    variance = [tf.zeros_like(tensor) for tensor in weights]

    # num_sample 개의 데이터 랜덤샘플링 
    indices = np.random.choice(len(data), size=num_sample, replace=False)

    for i in indices:

        with tf.GradientTape() as tape:
            tape.watch(weights)
            x = tf.expand_dims(data[i], axis=0)
            output = model(x, training=False) # (수정) 메모리 문제, 모든 데이터를 한번에 넣으면 오류 생김. 여기서는 하나씩 열개의 데이터를 사용 
            log_likelihood = tf.math.log(output)

        gradients = tape.gradient(log_likelihood, weights)
        variance = [var + (grad ** 2) for var, grad in zip(variance, gradients)]

    fisher_matrix = [tensor / num_sample for tensor in variance]
    
    return fisher_matrix

In [9]:
# (수정) 배치를 사용하지 않음 
def evaluate(model, test_set):
  acc = tf.keras.metrics.CategoricalAccuracy(name='accuracy')
  for i, (seq, labels) in enumerate(test_set):
    preds = model.predict_on_batch(seq)
    acc.update_state(labels, preds)
  return acc.result().numpy()

In [10]:

def train_loop(model, OPTIMIZER, data, test_size,
                first_task = 44, inc_task = 5, first_epochs = 30, inc_epochs = 5,
                  lamb=0, num_sample=10):
    
    first_part = split_by_label(data, 0, first_task)
    train, test = split_train_test(first_part, test_size=test_size, random_state=11)
    

    # OPTIMIZER -> param
    i = 0
    while(1):

        if ( first_task + i * inc_task ) <= MAX_LABEL:
            
            if i == 0:
                model.compile(loss=CategoricalCrossentropy(from_logits=False), optimizer=OPTIMIZER, metrics=["accuracy"])

                # 3D ndarray 로 변환 
                train_seq, train_label = split_data_label(train)

                train_seq = np.stack(train_seq.values)
                train_seq = train_seq[..., np.newaxis]

                train_label = train_label.values
                train_label = to_categorical(train_label, num_classes=MAX_LABEL)
                

                history = model.fit(x=train_seq, y=train_label, epochs=first_epochs, verbose=2)
                print(f"   First_task training accuracy: {history.history['accuracy'][-1]:.4f}")

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, train_seq, num_sample=num_sample)

                i = i + 1

            else:
                # 데이터 준비 
                inc_part = split_by_label(data, first_task + (i-1) * inc_task + 1, first_task + i * inc_task )
                train, inc_test = split_train_test(inc_part, test_size=test_size, random_state=11)

                model.compile(loss=ewc_loss(model, fisher_matrix, lamb=lamb), optimizer=OPTIMIZER, metrics=["accuracy"])
                
                # 3D ndarray 로 변환 (이부분 함수로 바꾸기)
                train_seq, train_label = split_data_label(train)

                train_seq = np.stack(train_seq.values)
                train_seq = train_seq[..., np.newaxis]

                train_label = train_label.values
                train_label = to_categorical(train_label, num_classes=MAX_LABEL)



                # train
                history = model.fit(x=train_seq, y=train_label, epochs=inc_epochs, verbose=2)
                print(f"   {i}_task training accuracy: {history.history['accuracy'][-1]:.4f}")



                # (수정) 일종의 전처리이므로 preprocessing 또는 utils에 함수 작성 
                # 축적된 test로 정확도 측정 (중요, EWC 성능)
                test_seq, test_label = split_data_label(test)
                test_seq = np.stack(test_seq.values)
                test_seq = test_seq[..., np.newaxis]

                test_label = test_label.values
                test_label = to_categorical(test_label, num_classes=MAX_LABEL)

                test_ = tf.data.Dataset.from_tensor_slices((test_seq, test_label))
                test_ = test_.batch(32) #(수정) 모델 자체 배치 존재? - 학습시 fit 디폴트값도 32

                inc_accuracy = evaluate(model, test_)
                print(f"Task {i} accuracy after training on Task ~{i-1}: {inc_accuracy:.4f}")




                # test 업데이트 
                test = accumulate_data(test, inc_test)

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, train_seq, num_sample=num_sample)

                i = i + 1

        else:
            break 

## Joint

In [25]:

def train_loop_joint(model, OPTIMIZER, data, test_size,
                first_task = 44, inc_task = 5, first_epochs = 30, inc_epochs = 5,
                  lamb=0, num_sample=10):
    
    first_part = split_by_label(data, 0, first_task)
    train, test = split_train_test(first_part, test_size=test_size, random_state=11)
    

    # OPTIMIZER -> param
    i = 0
    while(1):

        if ( first_task + i * inc_task ) <= MAX_LABEL:
            
            if i == 0:
                model.compile(loss=CategoricalCrossentropy(from_logits=False), optimizer=OPTIMIZER, metrics=["accuracy"])

                # 3D ndarray 로 변환 
                train_seq, train_label = split_data_label(train)

                train_seq = np.stack(train_seq.values)
                train_seq = train_seq[..., np.newaxis]

                train_label = train_label.values
                train_label = to_categorical(train_label, num_classes=MAX_LABEL)
                

                history = model.fit(x=train_seq, y=train_label, epochs=first_epochs, verbose=2)
                print(f"   First_task training accuracy: {history.history['accuracy'][-1]:.4f}")

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, train_seq, num_sample=num_sample)

                i = i + 1

            else:
                # 데이터 준비 
                inc_part = split_by_label(data, first_task + (i-1) * inc_task + 1, first_task + i * inc_task )
                inc_train, inc_test = split_train_test(inc_part, test_size=test_size, random_state=11)
                train = accumulate_data(train, inc_train)

                model.compile(loss=ewc_loss(model, fisher_matrix, lamb=lamb), optimizer=OPTIMIZER, metrics=["accuracy"])
                
                # 3D ndarray 로 변환 (이부분 함수로 바꾸기)
                train_seq, train_label = split_data_label(train)

                train_seq = np.stack(train_seq.values)
                train_seq = train_seq[..., np.newaxis]

                train_label = train_label.values
                train_label = to_categorical(train_label, num_classes=MAX_LABEL)



                # train
                history = model.fit(x=train_seq, y=train_label, epochs=inc_epochs, verbose=2)
                print(f"   {i}_task training accuracy: {history.history['accuracy'][-1]:.4f}")



                # (수정) 일종의 전처리이므로 preprocessing 또는 utils에 함수 작성 
                # 축적된 test로 정확도 측정 (중요, EWC 성능)
                test_seq, test_label = split_data_label(test)
                test_seq = np.stack(test_seq.values)
                test_seq = test_seq[..., np.newaxis]

                test_label = test_label.values
                test_label = to_categorical(test_label, num_classes=MAX_LABEL)

                test_ = tf.data.Dataset.from_tensor_slices((test_seq, test_label))
                test_ = test_.batch(32) #(수정) 모델 자체 배치 존재? - 학습시 fit 디폴트값도 32

                inc_accuracy = evaluate(model, test_)
                print(f"Task {i} accuracy after training on Task ~{i-1}: {inc_accuracy:.4f}")




                # test 업데이트 
                test = accumulate_data(test, inc_test)

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, train_seq, num_sample=num_sample)

                i = i + 1

        else:
            break 

## ce ratio

In [16]:
def ewc_loss_ce(model, fisher_matrix, lamb, alpha):
    optimal_weights = deepcopy(model.trainable_weights)

    def loss_fn(y_true, y_pred):

        ce_loss = alpha * CategoricalCrossentropy(from_logits=False)(y_true, y_pred)
        ewc_loss = compute_ewc_penalty(model, fisher_matrix, optimal_weights, lamb=lamb)

        return ce_loss + ewc_loss
    
    return loss_fn

In [17]:

def train_loop_ce(model, OPTIMIZER, data, test_size,
                first_task = 44, inc_task = 5, first_epochs = 30, inc_epochs = 5,
                  lamb=0, alpha=0, num_sample=10):
    
    first_part = split_by_label(data, 0, first_task)
    train, test = split_train_test(first_part, test_size=test_size, random_state=11)
    

    # OPTIMIZER -> param
    i = 0
    while(1):

        if ( first_task + i * inc_task ) <= MAX_LABEL:
            
            if i == 0:
                model.compile(loss=CategoricalCrossentropy(from_logits=False), optimizer=OPTIMIZER, metrics=["accuracy"])

                # 3D ndarray 로 변환 
                train_seq, train_label = split_data_label(train)

                train_seq = np.stack(train_seq.values)
                train_seq = train_seq[..., np.newaxis]

                train_label = train_label.values
                train_label = to_categorical(train_label, num_classes=MAX_LABEL)
                

                history = model.fit(x=train_seq, y=train_label, epochs=first_epochs, verbose=2)
                print(f"   First_task training accuracy: {history.history['accuracy'][-1]:.4f}")

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, train_seq, num_sample=num_sample)

                i = i + 1

            else:
                # 데이터 준비 
                inc_part = split_by_label(data, first_task + (i-1) * inc_task + 1, first_task + i * inc_task )
                train, inc_test = split_train_test(inc_part, test_size=test_size, random_state=11)

                model.compile(loss=ewc_loss(model, fisher_matrix, lamb=lamb, alpha=alpha), optimizer=OPTIMIZER, metrics=["accuracy"])
                
                # 3D ndarray 로 변환 (이부분 함수로 바꾸기)
                train_seq, train_label = split_data_label(train)

                train_seq = np.stack(train_seq.values)
                train_seq = train_seq[..., np.newaxis]

                train_label = train_label.values
                train_label = to_categorical(train_label, num_classes=MAX_LABEL)



                # train
                history = model.fit(x=train_seq, y=train_label, epochs=inc_epochs, verbose=2)
                print(f"   {i}_task training accuracy: {history.history['accuracy'][-1]:.4f}")



                # (수정) 일종의 전처리이므로 preprocessing 또는 utils에 함수 작성 
                # 축적된 test로 정확도 측정 (중요, EWC 성능)
                test_seq, test_label = split_data_label(test)
                test_seq = np.stack(test_seq.values)
                test_seq = test_seq[..., np.newaxis]

                test_label = test_label.values
                test_label = to_categorical(test_label, num_classes=MAX_LABEL)

                test_ = tf.data.Dataset.from_tensor_slices((test_seq, test_label))
                test_ = test_.batch(32) #(수정) 모델 자체 배치 존재? - 학습시 fit 디폴트값도 32

                inc_accuracy = evaluate(model, test_)
                print(f"Task {i} accuracy after training on Task ~{i-1}: {inc_accuracy:.4f}")




                # test 업데이트 
                test = accumulate_data(test, inc_test)

                # Fisher matrix 계산 
                fisher_matrix = compute_fisher_matrix(model, train_seq, num_sample=num_sample)

                i = i + 1

        else:
            break 

# test

In [11]:
data = pd.read_pickle('mon_data.pkl')
print(data.shape)
MAX_LABEL = 80

(19000, 2)


## outputs

## today

In [27]:
model = DFNet.build(input_shape=(10000, 1), classes=MAX_LABEL)
# 옵티마이저 설정 
OPTIMIZER = Adamax(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
train_loop_joint(model, OPTIMIZER, data, test_size=0.2, first_task = 39, inc_task = 10, first_epochs = 1, inc_epochs = 1, lamb=0.0, num_sample=100)

c:\ProgramData\Anaconda3\envs\naivis\lib\site-packages\keras\optimizers\optimizer_v2\adamax.py:99: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


200/200 - 13s - loss: 3.8934 - accuracy: 0.0708 - 13s/epoch - 67ms/step
   First_task training accuracy: 0.0708
250/250 - 18s - loss: 3.8948 - accuracy: 0.1074 - 18s/epoch - 73ms/step
   1_task training accuracy: 0.1074
Task 1 accuracy after training on Task ~0: 0.0219
300/300 - 21s - loss: 3.8765 - accuracy: 0.1266 - 21s/epoch - 70ms/step
   2_task training accuracy: 0.1266
Task 2 accuracy after training on Task ~1: 0.0440
350/350 - 24s - loss: 3.8438 - accuracy: 0.1442 - 24s/epoch - 68ms/step
   3_task training accuracy: 0.1442
Task 3 accuracy after training on Task ~2: 0.0737
400/400 - 26s - loss: 3.7137 - accuracy: 0.1667 - 26s/epoch - 66ms/step
   4_task training accuracy: 0.1667
Task 4 accuracy after training on Task ~3: 0.2318


In [23]:
# 모델 빌드 
model = DFNet.build(input_shape=(10000, 1), classes=MAX_LABEL)
# 옵티마이저 설정 
OPTIMIZER = Adamax(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
train_loop(model, OPTIMIZER, data, test_size=0.2, first_task = 39, inc_task = 10, first_epochs = 100, inc_epochs = 50, lamb=0.0, num_sample=100)

Epoch 1/100


c:\ProgramData\Anaconda3\envs\naivis\lib\site-packages\keras\optimizers\optimizer_v2\adamax.py:99: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


200/200 - 13s - loss: 3.8196 - accuracy: 0.0886 - 13s/epoch - 64ms/step
Epoch 2/100
200/200 - 11s - loss: 3.2640 - accuracy: 0.1450 - 11s/epoch - 55ms/step
Epoch 3/100
200/200 - 11s - loss: 3.0052 - accuracy: 0.1955 - 11s/epoch - 56ms/step
Epoch 4/100
200/200 - 11s - loss: 2.8152 - accuracy: 0.2294 - 11s/epoch - 56ms/step
Epoch 5/100
200/200 - 11s - loss: 2.6011 - accuracy: 0.2800 - 11s/epoch - 57ms/step
Epoch 6/100
200/200 - 12s - loss: 2.4323 - accuracy: 0.3244 - 12s/epoch - 58ms/step
Epoch 7/100
200/200 - 12s - loss: 2.3025 - accuracy: 0.3527 - 12s/epoch - 58ms/step
Epoch 8/100
200/200 - 12s - loss: 2.1510 - accuracy: 0.3998 - 12s/epoch - 58ms/step
Epoch 9/100
200/200 - 12s - loss: 2.0243 - accuracy: 0.4372 - 12s/epoch - 58ms/step
Epoch 10/100
200/200 - 12s - loss: 1.9064 - accuracy: 0.4661 - 12s/epoch - 58ms/step
Epoch 11/100
200/200 - 12s - loss: 1.7742 - accuracy: 0.5008 - 12s/epoch - 58ms/step
Epoch 12/100
200/200 - 12s - loss: 1.6462 - accuracy: 0.5411 - 12s/epoch - 58ms/step
E

KeyboardInterrupt: 